# 作業工程計画
2023.1.13作成<br>
1. 21年、22年夏、22年冬の採点者情報を使用。採点者品質予測_22冬納品_前処理済み学習データ_v2.xlsx
2. 学年情報と科目情報を削除し、科目別に分離
3. 設問情報を読み込み。科目別に学年と分野の組み合わせを作成
4. 22冬データに組み合わせ表を結合。これを予測用訓練データとする。
5. 21年と22夏データを読み込む。21年データを学習データに、22夏データを検証データとする。
6. 上記の二つのデータフレームを結合し、学習データ、検証データ、予測データのラベルをつける
7. モデルを作成し、予測値をリストとして出力

データ加工に使用しているスクリプト<br>
採点者品質予測_22冬納品_前処理_1226_V1.IPYNB<br>
採点者品質予測_22冬納品_機械学習_1226_V1.IPYNB<br>

reference:https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

# ライブラリ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import japanize_matplotlib
import seaborn as sns
import re 
import scipy as sp
import time

"""
機械学習ライブラリの準備
"""

from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report #2値分類評価指標を出力

from sklearn.tree import DecisionTreeRegressor #決定木
import lightgbm as lgb #lightGBM

from sklearn import tree
import graphviz

In [2]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

# 予測値リストの作成

## 前処理済みデータを読み込み,予測データに学年分野を組み合わせて追加

In [3]:
"""
Excelの読み込み
"""
name_file = "採点者品質予測_22冬納品_前処理済み学習データ_v2.xlsx" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
#sn_list ['整形済みデータ', '前処理済みデータ', '機械学習用データ_新人', '機械学習用データ_経験者', '機械学習用データ_etc']

In [4]:
# Excelファイルの読み込み
df_shinjin = pd.read_excel(path_file, sn_list[2], header=0, index_col=None,skiprows=None)
df_keiken = pd.read_excel(path_file, sn_list[3], header=0, index_col=None,skiprows=None)
df_etc = pd.read_excel(path_file, sn_list[4], header=0, index_col=None,skiprows=None)

In [5]:
df_keiken = df_keiken.rename(columns = {'今回の分野':'分野'})

In [6]:
df_shinjin['年度'].unique()

array([2021, 2022])

In [305]:
len(df_shinjin['data']==2)+len(df_keiken['data']==2)+len(df_etc['data']==2)

13850

In [7]:
def remake_bunya(df_input):
    df = df_input.copy()
    #22冬データを選択
    df_res = df[~(df['data']==2)]
    df = df[df['data']==2]
    #科目別に抽出
    df_jpn = df[df['科目']=='国語']
    df_math= df[df['科目']=='数学']
    df_eng = df[df['科目']=='英語']
    dfs = [df_jpn, df_math, df_eng]
    for i in dfs:
        #分野と学年のユニーク値を取得し、データフレーム化して結合キーを付加
        df_bunya=pd.DataFrame({'分野':i['分野'].unique()})
        df_bunya['join_key']=0
        df_gakunen=pd.DataFrame({'学年':i['学年'].unique()})
        df_gakunen['join_key']=0
        #全結合 crossjoinを行う
        df_bunya_gakunen = pd.merge(df_bunya, df_gakunen,  on="join_key")
        df_bunya_gakunen['join_key']=0
        #元の分野を削除し、結合キーを付加
        df_temp = i.drop(columns = {'分野','学年'})
        df_temp['join_key']=0
        #全結合 crossjoinを行う
        df_test = pd.merge(df_temp, df_bunya_gakunen,  on="join_key")
        df_test = df_test.drop(columns = 'join_key')
        df_res = df_res.append(df_test)
    return df_res

In [8]:
df_pret_shinjin = remake_bunya(df_shinjin)
df_pret_keiken = remake_bunya(df_keiken)
df_pret_etc = remake_bunya(df_etc)

In [9]:
df_pret_shinjin['年度'].unique()

array([2021, 2022])

In [10]:
"""
Excelの書き出し 
"""
name = "配置最適化_前処理済み学習データ_v2"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df1 = df_pret_shinjin
df2 = df_pret_keiken
df3 = df_pret_etc

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df1.to_excel(writer, sheet_name='機械学習用データ_新人',encoding='utf-8-sig', index = False)
    df2.to_excel(writer, sheet_name='機械学習用データ_経験者',encoding='utf-8-sig', index = False)
    df3.to_excel(writer, sheet_name='機械学習用データ_etc',encoding='utf-8-sig', index = False)

## 予測値の作成

### DecisionTreeRegressorメソッドの概要

|引数名|概要|デフォルト|
| :---- | :---- | :---- |
|criterion|不純度を測定する基準（平均二乗誤差、平均絶対誤差など）|‘mse’|
|splitter|条件探索アルゴリズムを選択するオプション（’best’と’rondom’が指定可能）|‘best’|
|max_depth|決定木のノード深さの制限値。ツリーが深くなりすぎて過学習の状態に陥った際は、このパラメータが正則化の役割を果たす。|None|
|min_samples_split|ノードを分割するために必要なサンプルの最小値|2|
|min_samples_leaf|1ノードの深さを作成するために必要となるデータ数の最小値。指定した値以上のデータ数を持たないノードは作られない。|1|
|min_weight_fraction_leaf|サンプルの重みを考慮した上でのmin_samples_leafに該当|0.0|
|max_features|ランダムに指定する説明変数の数(全ての説明変数がモデル学習に活用されるわけではなく、ランダムに割り振られる）|None|
|random_state|乱数シード|None|
|max_leaf_nodes|作成される決定木の葉の数を、指定した値以下に制御する|None|
|min_impurity_decrease|決定木の成長の早期停止するための閾値。不純度が指定の値より減少した場合、ノードを分岐し、不純度が指定の値より減少しなければ分岐を抑制。|0.0|
|ccp_alpha|ccp_alphaが大きいほどプルーニングされるノードの数が増加。プルーニングとは、精度低下をできるだけ抑えながら過剰な重みを排除するプロセスを指す。|0.0|


In [11]:
"""
https://stackoverflow.com/questions/50607740/reverse-a-get-dummies-encoding-in-pandas
ダミー変数を元に戻す。エクセル出力用
"""

def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [12]:
#機械学習に使うデータの前処理_新人用
df_pret_shinjin.columns.values

array(['スタッフコード', '参加回数', '科目', '分野', '年度', '採点回', '学年', '完全一致率',
       '登録試験点数', '身分', '年齢', '偏差値', 'data'], dtype=object)

In [24]:
def make_data_shinjin(df_input):
    
    """
    学習,検証,予測データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    df_pred = df[df['data']==2].copy()
    
    #rem_y = '参加回数'
    use_y = '完全一致率'

    #1.不要な変数を削除
    #df = df.drop(columns = rem_y)

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, columns=['分野','身分'])
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = use_y)#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,[use_y,'data']]#目的変数と選択用変数だけ含むデータフレーム　
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[df_X['data']==0]
    #訓練用目的変数
    y_train = df_y[df_y['data']==0]

    """22夏が検証用"""
    #説明変数
    X_test = df_X[df_X['data']==1]
    #目的変数
    y_test = df_y[df_y['data']==1]

    """予測用"""
    #説明変数
    X_pred = df_X[df_X['data']==2]#秋に変更するところ
    
    #5.不要な変数を削除
    rem_cols_x = ["スタッフコード","採点回","年度","科目","参加回数","data"]
    rem_cols_y = ["data"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_id =X_test.loc[:,['スタッフコード']]
    X_test_kamoku =X_test.loc[:,['科目']]
    X_test = X_test.drop(columns = rem_cols_x)
    X_pred = X_pred.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku, X_pred, df_pred]

    return df_res

In [14]:
def make_data_shinjin(df_input):#オリジナル
    
    """
    学習,検証,予測データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    #0の場合、完全一致率を予測。　1の場合、最終ペース相対値を予測
    rem_y = '参加回数'
    use_y = '完全一致率'

    #1.不要な変数を削除
    df = df.drop(columns = rem_y)

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = use_y)#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,[use_y,'data']]#目的変数と選択用変数だけ含むデータフレーム　
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==2022))]#秋に変更するところ
    #訓練用目的変数
    y_train = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==2022))]#秋に変更するところ

    """22夏が検証用"""
    #訓練用説明変数
    X_test = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==2022)]#秋に変更するところ
    #訓練用目的変数
    y_test = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==2022)]#秋に変更するところ
    
    #5.不要な変数を削除
    rem_cols_x = ["スタッフコード","採点回","年度","科目","参加回数","data"]
    rem_cols_y = ["data"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_id =X_test.loc[:,['スタッフコード']]
    X_test_kamoku =X_test.loc[:,['科目']]
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku]

    return df_res

In [15]:
df_pret_keiken.columns.values

array(['スタッフコード', '前回の参加回数', '前回の分野', '前回の完全一致率', '科目', '今回の参加回数', '分野',
       '今回の年度', '今回の採点回', '学年', '今回の完全一致率', '登録試験点数', '身分', '年齢', '偏差値',
       'data'], dtype=object)

In [25]:
def make_data_keiken(df_input):
    
    """
    学習,検証,予測データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    df_pred = df[df['data']==2].copy()
    df_pred = df_pred.rename(columns = {'今回の採点回':'採点回',#0始まりで数えているため、
                             '今回の年度':'年度',
                             '今回の参加回数':'参加回数',
                             '今回の分野':'分野'})
    rem_y = ['前回の分野','前回の参加回数']
    use_y = '今回の完全一致率'

    
    #1.不要な変数を削除
    df = df.drop(columns = rem_y)
    df = df.rename(columns = {'今回の採点回':'採点回',#0始まりで数えているため、
                             '今回の年度':'年度',
                             '今回の参加回数':'参加回数',
                             '今回の分野':'分野'})

    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, columns=['分野','身分'])
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = use_y)#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,['data',use_y]]#目的変数と選択用変数だけ含むデータフレーム　#秋に変更するところ
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[df_X['data']==0]#秋に変更するところ
    #訓練用目的変数
    y_train = df_y[df_y['data']==0]#秋に変更するところ

    """22夏が検証用"""
    #説明変数
    X_test = df_X[df_X['data']==1]#秋に変更するところ
    #目的変数
    y_test = df_y[df_y['data']==1]#秋に変更するところ

    """予測用"""
    #説明変数
    X_pred = df_X[df_X['data']==2]#秋に変更するところ
    
    #5.不要な変数を削除
    rem_cols_x = ["スタッフコード","採点回","年度","科目","data"]
    rem_cols_y = ["data"]#秋に変更するところ

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_id =X_test.loc[:,['スタッフコード']]
    X_test_kamoku =X_test.loc[:,['科目']]
    X_test = X_test.drop(columns = rem_cols_x)
    X_pred = X_pred.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku, X_pred, df_pred]

    return df_res

In [17]:
"""
モデル作成
"""
def train_model(df,a=2,b=1,c=None):
    X_train= df[0]
    y_train= df[1]
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=0,#出力結果の固定のため 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    return model

In [18]:
def get_test(df,model):
    """
    モデルを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
    """
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    X_test_id = df[4]
    X_test_kamoku = df[5]
    y_pred  = model.predict(X_test)
    
    #得た結果を学習データとマージしてデータフレームで返す
    
    df_test=[]
    df_test = undummify(X_test)  #企画ペースを入れるためにここをいじった。元はX_test
    df_test.loc[:,'スタッフコード']= X_test_id
    df_test.loc[:,'科目']= X_test_kamoku
    df_test.loc[:,'完全一致率']= y_test
    df_test.loc[:,'AI想定完全一致率']= y_pred #上のデータに予測値をマージ
    #学習データの同じ学年分野カテゴリの中の基礎統計量
    df_train = undummify(X_train)
    df_train.loc[:,'完全一致率'] = y_train
    df_temp = df_train.groupby(['分野','学年'],as_index=False).agg(
        学習データ_N数=pd.NamedAgg(column="完全一致率", aggfunc="count"),
        学習データ_平均値=pd.NamedAgg(column="完全一致率", aggfunc="mean"),
        学習データ_中央値=pd.NamedAgg(column="完全一致率", aggfunc="median"),
        学習データ_最大値=pd.NamedAgg(column="完全一致率", aggfunc="max"),
        学習データ_最小値=pd.NamedAgg(column="完全一致率", aggfunc="min"))
    df_res = pd.merge(df_test, df_temp, on=['分野', '学年'], how='left')
    # 実測値_完全一致率をランク分け
    x = "完全一致率"
    conditions = [
            (df_res[x] >= 0.95),
            (df_res[x] >= 0.7)
             ]
    choices = ["0.95~","0.70~0.95"]
    df_res.loc[:,'実測一致率ランク'] = np.select(conditions, choices, default = "~0.70")

        # AI想定完全一致率をランク分け 評価用
    x = "AI想定完全一致率"
    conditions = [
            (df_res[x] >= 0.95),
            (df_res[x] >= 0.7)
             ]
    choices = ["0.95~","0.70~0.95"]
    df_res.loc[:,'想定一致率ランク'] = np.select(conditions, choices, default = "~0.70")
    
    return df_res

In [19]:
def get_pred(df,model):
    """
    モデルを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
    """
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    X_test_id = df[4]
    X_test_kamoku = df[5]
    X_pred = df[6]
    df_pred = df[7]
    y_pred  = model.predict(X_pred)
    
    #得た結果を学習データとマージしてデータフレームで返す
    
    df_test=[]
    df_test = undummify(df_pred)  #企画ペースを入れるためにここをいじった。元はX_test
    df_test.loc[:,'AI想定完全一致率']= y_pred #上のデータに予測値をマージ
    #学習データの同じ学年分野カテゴリの中の基礎統計量
    df_train = undummify(X_train)
    df_train.loc[:,'完全一致率'] = y_train
    df_temp = df_train.groupby(['分野','学年'],as_index=False).agg(
        学習データ_N数=pd.NamedAgg(column="完全一致率", aggfunc="count"),
        学習データ_平均値=pd.NamedAgg(column="完全一致率", aggfunc="mean"),
        学習データ_中央値=pd.NamedAgg(column="完全一致率", aggfunc="median"),
        学習データ_最大値=pd.NamedAgg(column="完全一致率", aggfunc="max"),
        学習データ_最小値=pd.NamedAgg(column="完全一致率", aggfunc="min"))
    df_res = pd.merge(df_test, df_temp, on=['分野', '学年'], how='left')
    
    return df_res

In [20]:
#検証用
def get_result_test(df_input,case=int):
    df = df_input.copy()
    df_temp=[]#[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku]
    model = []
    pred = []
    #新人モデルの結果
    if case == 0:
        df_temp = make_data_shinjin(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_test(df_temp, model)
    #経験者モデルの結果
    elif case == 1:
        df_temp = make_data_keiken(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_test(df_temp, model)
            
    return pred

In [21]:
#予測用
def get_result_pred(df_input,case=int):
    df = df_input.copy()
    df_temp=[]#[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku]
    model = []
    pred = []
    #新人モデルの結果
    if case == 0:
        df_temp = make_data_shinjin(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_pred(df_temp, model)
    #経験者モデルの結果
    elif case == 1:
        df_temp = make_data_keiken(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_pred(df_temp, model)
            
    return pred

## 予測値の出力

In [ ]:
df1_pred = get_result_pred(df_pret_shinjin, case = 0)#新人モデルの予測結果
df2_pred = get_result_pred(df_pret_keiken, case = 1)#経験モデルの予測結果
df3_pred = get_result_pred(df_pret_etc, case = 0)#経験（過去データなし）モデルの予測結果

In [185]:
def select_columns(df_input):
    df = df_input.copy()
    collist=['スタッフコード','科目', '分野','学年','AI想定完全一致率']
    df = df[collist]
    return df

In [30]:
def get_dfs(df_input):
    df1 = select_columns(df_input[0])
    df2 = select_columns(df_input[1])
    df3 = select_columns(df_input[2])
    df_res = df1.append(df2)
    df_res = df_res.append(df3)
    return df_res    

In [63]:
name_file = "配置最適化_学年分野別予測値リスト_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

In [186]:
name_file = "配置最適化_学年分野別予測値リスト_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

#結果出力
dfs = [df1_pred, df2_pred, df3_pred]
df = get_dfs(dfs)

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df.to_excel(writer, sheet_name='予測値',encoding='utf-8-sig', index = False)

In [187]:
dfs = [df1_pred, df2_pred, df3_pred]
df_predlist = get_dfs(dfs)

# 配属優先度と希望人数リストの作成

### 希望人数

In [246]:
"""
配置希望人数を読み込み
"""
name_file = "22冬採点者配置希望人数.xlsx" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/rawdata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
#sn_list ['整形済みデータ', '前処理済みデータ', '機械学習用データ_新人', '機械学習用データ_経験者', '機械学習用データ_etc']

df_eng = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=[1])#headerを修正 ,usecols = [0,1,2,3,4]
df_jpn = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=[1])#headerを修正
df_math = pd.read_excel(path_file, sn_list[2], header=0, index_col=None,skiprows=None)#headerを修正
df_math['科目'] = df_math['科目'].replace(['数学X', '数学Y', '数学Z','数学X/A\u3000', '数学Y/B', '数学X/A・Y/B', '数学X/A'], '数学')

In [247]:
"""
分野マスタを読み込み
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/分野区分マスタv4.1.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

df_bunya = []
df_bunya = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None)#headerを修正
df_bunya = df_bunya.drop(['科目コード','ポイント採点', '解答言語'], axis=1)
#bunya = bunya.fillna({'解答言語': '日本語'})
df_bunya = df_bunya.rename(columns={'分野名':'分野修正前'})

In [248]:
#国語、数学、英語のデータフレームを結合し、分野名を修正し、学年と分野でグループし、配置希望人数を合計したフレームを返す。
def make_number(df_jpn, df_math, df_eng, df_bunya):
    collist=['年度', '採点回', '学年', '科目', 'ロット番号','分野','配置希望人数']
    df1=df_jpn[collist]
    df2=df_math[collist]
    df3=df_eng[collist]
    df =[]
    df = df1.append(df2)
    df = df.append(df3)
    df = df.rename(columns={'分野':'分野修正前','ロット番号':'割当'})
    df = df.round({'配置希望人数': 0})
    #分野の表記を修正
    df_join = pandas_tool.merge(df,df_bunya, key = ["分野修正前"],how = 'left')
    df_ret = df_join.groupby(['科目','分野','学年'],as_index=False).agg(配置希望人数=pd.NamedAgg(column="配置希望人数", aggfunc="sum"))
    return df_ret

In [249]:
df_number = make_number(df_jpn, df_math, df_eng, df_bunya)

df1のキー重複
36
df2のキー重複
0
df1のオブザベーション:59
df2のオブザベーション:164
mergeのオブザベーション:59


In [250]:
#結果出力
name_file = "配置最適化_学年分野配属人数リスト_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

#結果出力
df = df_number

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

### 配属優先度

In [252]:
def make_priority(df_input1, df_input2, df_input3):
    df1 = df_input1.copy()
    df2 = df_input2.copy()
    df3 = df_input3.copy()
    dfs = df1.append(df2)
    dfs = dfs.append(df3)
    dfs = dfs[~(dfs['data']==2)]
    collist=['スタッフコード','科目', '分野','学年','完全一致率']
    dfs=dfs[collist]
    dfs['科目'] = dfs['科目'].replace(['数学X', '数学Y', '数学Z','数学X/A\u3000', '数学Y/B', '数学X/A・Y/B', '数学X/A'], '数学')
    #分野の表記を修正
    dfs = dfs.rename(columns={'分野':'分野修正前'})
    df_join = pandas_tool.merge(dfs,df_bunya, key = ["分野修正前"],how = 'left')
    #平均値の集計
    df_join = df_join.groupby(['科目','分野','学年'],as_index=False).agg(平均完全一致率=pd.NamedAgg(column="完全一致率", aggfunc="mean"))
    
    #列選択
    collist=['科目', '分野','学年','平均完全一致率']
    df = df_join[collist]
    df_jpn = df[df['科目']=='国語']
    df_jpn.loc[:, '優先度_修正前'] = df_jpn['平均完全一致率'].rank(method='min', ascending=True)
    df_math = df[df['科目']=='数学']
    df_math.loc[:, '優先度_修正前'] = df_math['平均完全一致率'].rank(method='min', ascending=True)
    df_eng = df[df['科目']=='英語']
    df_eng.loc[:, '優先度_修正前'] = df_eng['平均完全一致率'].rank(method='min', ascending=True)
    df_ret = [df, df_jpn, df_math, df_eng]
    return df_ret

In [253]:
df_priority_bef = make_priority(df_shinjin, df_keiken, df_etc)

df1のキー重複
10025
df2のキー重複
0
df1のオブザベーション:10057
df2のオブザベーション:164
mergeのオブザベーション:10057


In [154]:
#結果出力
name_file = "配置最適化_学年分野優先度リスト_修正前_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

#結果出力
df = df_priority_bef

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[0].to_excel(writer, sheet_name='全科目',encoding='utf-8-sig', index = False)
    df[1].to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df[2].to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df[3].to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)

### dfの結合

In [254]:
"""
配属優先度を読み込み
"""
name_file = "配置最適化_学年分野優先度リスト_修正後_v1.xlsx" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

df1 = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=None)#国語
df2 = pd.read_excel(path_file, sn_list[2], header=0, index_col=None,skiprows=None)#数学
df3 = pd.read_excel(path_file, sn_list[3], header=0, index_col=None,skiprows=None)#英語
df_priority_aft=[df1, df2, df3]

In [327]:
#分野の表記を修正
def merge_number_priority(df_input1, df_number):
    df1 = df_input1.copy()
    #優先度と配属人数の結合
    df_jpn = pandas_tool.merge(df1[0], df_number, key = ["科目","分野","学年"],how = 'left')
    df_math = pandas_tool.merge(df1[1], df_number, key = ["科目","分野","学年"],how = 'left')
    df_eng = pandas_tool.merge(df1[2], df_number, key = ["科目","分野","学年"],how = 'left')
    #配属人数が空欄の行を削除。つまり今回の設問にない分野を削除
    df_jpn = df_jpn.dropna(subset=['配置希望人数'])
    df_math = df_math.dropna(subset=['配置希望人数'])
    df_eng = df_eng.dropna(subset=['配置希望人数'])
    #優先度を修正
    df_jpn.loc[:, '優先度'] = df_jpn['優先度_修正後'].rank(method='min', ascending=True)
    df_math.loc[:, '優先度'] = df_math['優先度_修正後'].rank(method='min', ascending=True)
    df_eng.loc[:, '優先度'] = df_eng['優先度_修正後'].rank(method='min', ascending=True)
    #優先度で並び替え
    df_jpn= df_jpn.sort_values('優先度', ascending=True).reset_index(drop=True)
    df_math= df_math.sort_values('優先度', ascending=True).reset_index(drop=True)
    df_eng= df_eng.sort_values('優先度', ascending=True).reset_index(drop=True)
    #列選択
    collist=['科目', '分野','学年','配置希望人数','優先度']
    df_jpn = df_jpn[collist]
    df_math = df_math[collist]
    df_eng = df_eng[collist]
    #データタイプの変換
    df_jpn = df_jpn.astype({'配置希望人数': int})
    df_math = df_math.astype({'配置希望人数': int})
    df_eng = df_eng.astype({'配置希望人数': int})
    
    return [df_jpn, df_math, df_eng]

In [288]:
df_prilist = merge_number_priority(df_priority_aft, df_number)

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:20
df2のオブザベーション:35
mergeのオブザベーション:20
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:29
df2のオブザベーション:35
mergeのオブザベーション:29
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:17
df2のオブザベーション:35
mergeのオブザベーション:17


In [240]:
df_prilist[0].dtypes

科目         object
分野         object
学年          int64
配置希望人数      int64
優先度       float64
dtype: object

In [289]:
#結果出力
name_file = "配置最適化_学年分野優先度配属人数リスト_v2" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

#結果出力
df = df_prilist

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[0].to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df[1].to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df[2].to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)

# 最適化アルゴリズム構築

## データマートの確認

In [254]:
"""
予測値リストを読み込み
"""
name_file = "配置最適化_学年分野別予測値リスト_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

df = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=None)
df_predlist = df.copy()

In [400]:
"""
配属優先度を読み込み
"""
name_file = "配置最適化_学年分野優先度配属人数リスト_v2.xlsx" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

df1 = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None)#国語
df2 = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=None)#数学
df3 = pd.read_excel(path_file, sn_list[2], header=0, index_col=None,skiprows=None)#英語
df_prilist=[df1, df2, df3]

In [402]:
#優先度リスト 国語
df_prilist[0].head()

,科目,分野,学年,配置希望人数,優先度
0,国語,評論,2,190,1
1,国語,評論,1,205,2
2,国語,小説,1,154,3
3,国語,小説,2,190,4
4,国語,古文(内説),1,87,5


In [403]:
#予測リスト
df_predlist.head()

,スタッフコード,科目,分野,学年,AI想定完全一致率
0,1060604673,国語,古文(内説),1,0.833333
1,1060604673,国語,古文(内説),2,0.833333
2,1060604673,国語,古文(現訳),1,0.833333
3,1060604673,国語,古文(現訳),2,0.833333
4,1060604673,国語,小説,1,0.821429


In [299]:
#予測リスト
df_predlist.columns.values

array(['スタッフコード', '科目', '分野', '学年', 'AI想定完全一致率'], dtype=object)

## アルゴリズム1
優先度の高い分野から必要人数ずつ割当てていくケース

In [337]:
def alg1(df_predlist, df_prilist):
    """
    優先度リストから、優先度の高い分野から順に、分野名と人数を参照する for enumerate
    予測値リストから、分野名でフィルタリングし、予測値でソートをして、高い順に必要人数を選択
    結果を学年分野で集計し、優先度リストに結合
    """
    df_pred = df_predlist.copy()#予測値リスト
    df_pri = df_prilist.copy()
    df_res = pd.DataFrame(columns=['スタッフコード', '科目', '分野', '学年', 'AI想定完全一致率'])

    #科目を選択
    kamoku =[0,1,2]
    for k in kamoku:#科目を選択
        for idx, value in enumerate(df_pri[k]['優先度']):#分野名を選択
            # print(idx)
            bunya = df_pri[k].at[idx,'分野']#分野
            gakunen = df_pri[k].at[idx,'学年']#学年
            ninzu = df_pri[k].at[idx,'配置希望人数']#配置希望人数
            # print(bunya, gakunen, ninzu)
            
            #予測値リストを優先度に従って分野名と学年でフィルタ
            df_temp = df_pred[(df_pred['分野']==bunya)&(df_pred['学年']==gakunen)]
            #予測値を降順で並び替え
            df_temp = df_temp.sort_values('AI想定完全一致率', ascending=False)
            #配置希望人数を取得
            df_temp =df_temp.iloc[:ninzu,:]
            #選択結果を結果のデータフレームに保存
            df_res = df_res.append(df_temp)
            #取得したスタッフコードを削除
            selected_staff = pd.Series(df_temp['スタッフコード'].unique())
            df_pred = df_pred[~(df_pred['スタッフコード'].isin(selected_staff))]#選択されたスタッフコードを削除
    
    #結果の集計
    col_name = 'AI想定完全一致率'
    df_agg = df_res.groupby(['分野','学年'],as_index=False).agg(
        AI配置人数=pd.NamedAgg(column = col_name, aggfunc="count"),
        AI想定完全一致率_平均値=pd.NamedAgg(column = col_name, aggfunc="mean"),
        AI想定完全一致率_中央値=pd.NamedAgg(column = col_name, aggfunc="median"),
        AI想定完全一致率_最大値=pd.NamedAgg(column = col_name, aggfunc="max"),
        AI想定完全一致率_最小値=pd.NamedAgg(column = col_name, aggfunc="min"))

    #優先度と配属人数の結合
    df_jpn = pandas_tool.merge(df_pri[0], df_agg, key = ["分野","学年"],how = 'left')
    df_math = pandas_tool.merge(df_pri[1], df_agg, key = ["分野","学年"],how = 'left')
    df_eng = pandas_tool.merge(df_pri[2], df_agg, key = ["分野","学年"],how = 'left')
    
    return [df_res, df_jpn, df_math, df_eng]

In [338]:
alg1_result = alg1(df_predlist, df_prilist)

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:11
df2のオブザベーション:34
mergeのオブザベーション:11
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:16
df2のオブザベーション:34
mergeのオブザベーション:16
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:8
df2のオブザベーション:34
mergeのオブザベーション:8


In [339]:
#結果出力
name_file = "配置最適化_結果_アルゴリズム_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

#結果出力
df = alg1_result

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[0].to_excel(writer, sheet_name='alg1',encoding='utf-8-sig', index = False)
    df[1].to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df[2].to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df[3].to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)

## アルゴリズム2

In [423]:
import random
random.seed(0)
def alg2(df_predlist, df_prilist):
    """   
    学年分野別優先度と人数リストを読み込み
    学年分野別予測値リストを読み込んで、科目別に分割してデータフレームに入れる
    科目別に選択して回す
    分野に希望人数以上配置しないように、処理用の列を作る。
    """
    df_pris = df_prilist.copy()
    
    #df_predlistを科目別に分割し、df_predsとして、[国語、数学、英語]で作成。リストの中にデータフレーム
    df_pred_jpn = df_predlist[df_predlist['科目']=='国語'].copy()#予測値リスト
    df_pred_math = df_predlist[df_predlist['科目']=='数学'].copy()#予測値リスト
    df_pred_eng = df_predlist[df_predlist['科目']=='英語'].copy()#予測値リスト
    df_preds = [df_pred_jpn, df_pred_math, df_pred_eng]
    
    df_res = pd.DataFrame(columns=['スタッフコード', '科目', '分野', '学年', 'AI想定完全一致率'])
    
    #for文 0,1,2で国語、数学、英語を科目別に取得して回す
    kamoku =[0,1,2]
    for k in kamoku:
        df_pri = df_pris[k].copy() #優先度リスト
        #df_priにAI配置人数_cntという列を追加。初期値は配置希望人数
        df_pri['AI配置人数_cnt'] = df_pri['配置希望人数']
        df_pred = df_preds[k] #予測値リスト
        N = len(df_pred['スタッフコード'].unique())
        while N > 0:
            #優先度リストから、AI配置人数が0の分野を除外
            df_pri = df_pri[df_pri['AI配置人数_cnt'] != 0]
            df_pri = df_pri.reset_index(drop=True)
            # yusend_list = df_pri['優先度'].unique().tolist().sort(reverse=False)
            yusend_list = sorted(df_pri['優先度'].unique().tolist())
            for i in yusend_list:
                #df_priからランダムに取得した優先度の値を持つレコードの分野と学年の値を取得
                idx = df_pri.query('優先度 == @i').index[0]
                bunya = df_pri.at[idx,'分野']#分野
                gakunen = df_pri.at[idx,'学年']#学年
                ninzu = df_pri.at[idx,'AI配置人数_cnt']#配置希望人数
                df_pri.at[idx,'AI配置人数_cnt'] = ninzu -1#AI配置人数_cntから1を引く.0になった分野はもう選択しない。
                #候補者リストから分野と学年に一致するレコードを取得し、df_tempに追加
                df_temp = df_pred[(df_pred['分野']==bunya)&(df_pred['学年']==gakunen)]
                #予測値を降順で並び替え
                df_temp = df_temp.sort_values('AI想定完全一致率', ascending=False)
                #先頭を取得
                df_temp = df_temp[0:1]
                #選択結果を結果のデータフレームに保存
                df_res = df_res.append(df_temp)
                #取得したスタッフコードを削除
                selected_staff = pd.Series(df_temp['スタッフコード'].unique())
                df_pred = df_pred[~(df_pred['スタッフコード'].isin(selected_staff))]#選択されたスタッフコードを削除
                if len(df_pred)==0:#候補者全員を配置し終わった
                    break
            N -= len(yusend_list)
            
    #結果の集計
    col_name = 'AI想定完全一致率'
    df_agg = df_res.groupby(['分野','学年'],as_index=False).agg(
        AI配置人数=pd.NamedAgg(column = col_name, aggfunc="count"),
        予測値_平均値=pd.NamedAgg(column = col_name, aggfunc="mean"),
        予測値_中央値=pd.NamedAgg(column = col_name, aggfunc="median"),
        予測値_最大値=pd.NamedAgg(column = col_name, aggfunc="max"),
        予測値_最小値=pd.NamedAgg(column = col_name, aggfunc="min"))

    #優先度と配属人数の結合
    df_jpn = pandas_tool.merge(df_pris[0], df_agg, key = ["分野","学年"],how = 'left')
    df_math = pandas_tool.merge(df_pris[1], df_agg, key = ["分野","学年"],how = 'left')
    df_eng = pandas_tool.merge(df_pris[2], df_agg, key = ["分野","学年"],how = 'left')
    
    return [df_res, df_jpn, df_math, df_eng]

In [424]:
alg2_result = alg2(df_predlist, df_prilist)

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:11
df2のオブザベーション:35
mergeのオブザベーション:11
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:16
df2のオブザベーション:35
mergeのオブザベーション:16
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:8
df2のオブザベーション:35
mergeのオブザベーション:8


In [425]:
#結果出力
name_file = "配置最適化_結果_アルゴリズム2_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

#結果出力
df = alg2_result

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[0].to_excel(writer, sheet_name='ctr',encoding='utf-8-sig', index = False)
    df[1].to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df[2].to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df[3].to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)

## コントロール

In [401]:
import random
random.seed(0)
def ctr(df_predlist, df_prilist):
    """   
    """
    df_pris = df_prilist.copy()
    
    #df_predlistを科目別に分割し、df_predsとして、[国語、数学、英語]で作成。リストの中にデータフレーム
    df_pred_jpn = df_predlist[df_predlist['科目']=='国語'].copy()#予測値リスト
    df_pred_math = df_predlist[df_predlist['科目']=='数学'].copy()#予測値リスト
    df_pred_eng = df_predlist[df_predlist['科目']=='英語'].copy()#予測値リスト
    df_preds = [df_pred_jpn, df_pred_math, df_pred_eng]
    
    df_res = pd.DataFrame(columns=['スタッフコード', '科目', '分野', '学年', 'AI想定完全一致率'])
    
    #for文 0,1,2で国語、数学、英語を科目別に取得して回す
    kamoku =[0,1,2]
    for k in kamoku:
        df_pri = df_pris[k].copy() #優先度リスト
        #df_priにAI配置人数_cntという列を追加。初期値は配置希望人数で埋める. 
        df_pri['AI配置人数_cnt'] = df_pri['配置希望人数']
        df_pred = df_preds[k] #予測値リスト
        list_temp = list(df_pred['スタッフコード'].unique())
        random_id = random.sample(list_temp, len(list_temp))#予測値リストが含むスタッフコードをランダムに並べたリスト
        #df_random_idに該当するレコード（採点者）をdf_predから順番に取得(df_candidate)
        for r in random_id:
            df_candidate = df_pred[df_pred['スタッフコード'] == r]#ランダムに選んだ候補者
            
            #優先度リストから、AI配置人数が0の分野を除外
            df_pri = df_pri[df_pri['AI配置人数_cnt'] != 0]
            df_pri = df_pri.reset_index(drop=True)
            
            #df_priからランダムに取得した優先度の値を持つレコードの分野と学年の値を取得
            yusendo = random.choice(df_pri['優先度'].unique())
            idx = df_pri.query('優先度 == @yusendo').index[0]
            bunya = df_pri.at[idx,'分野']#分野
            gakunen = df_pri.at[idx,'学年']#学年
            ninzu = df_pri.at[idx,'AI配置人数_cnt']#配置希望人数
            
            #AI配置人数_cntから1を引く.0になった分野はもう選択しない。
            df_pri.at[idx,'AI配置人数_cnt'] = ninzu -1
            
            #候補者リストから分野と学年に一致するレコードを取得し、df_tempに追加
            df_temp = df_candidate[(df_candidate['分野']==bunya)&(df_candidate['学年']==gakunen)]
            #選択結果を結果のデータフレームに保存
            df_res = df_res.append(df_temp)
    
    #結果の集計
    col_name = 'AI想定完全一致率'
    df_agg = df_res.groupby(['分野','学年'],as_index=False).agg(
        AI配置人数=pd.NamedAgg(column = col_name, aggfunc="count"),
        予測値_平均値=pd.NamedAgg(column = col_name, aggfunc="mean"),
        予測値_中央値=pd.NamedAgg(column = col_name, aggfunc="median"),
        予測値_最大値=pd.NamedAgg(column = col_name, aggfunc="max"),
        予測値_最小値=pd.NamedAgg(column = col_name, aggfunc="min"))

    #優先度と配属人数の結合
    df_jpn = pandas_tool.merge(df_pris[0], df_agg, key = ["分野","学年"],how = 'left')
    df_math = pandas_tool.merge(df_pris[1], df_agg, key = ["分野","学年"],how = 'left')
    df_eng = pandas_tool.merge(df_pris[2], df_agg, key = ["分野","学年"],how = 'left')
    
    return [df_res, df_jpn, df_math, df_eng]

In [404]:
ctr_result = ctr(df_predlist, df_prilist)

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:11
df2のオブザベーション:35
mergeのオブザベーション:11
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:16
df2のオブザベーション:35
mergeのオブザベーション:16
df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:8
df2のオブザベーション:35
mergeのオブザベーション:8


In [405]:
#結果出力
name_file = "配置最適化_結果_コントロール_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

#結果出力
df = ctr_result

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[0].to_excel(writer, sheet_name='ctr',encoding='utf-8-sig', index = False)
    df[1].to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df[2].to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df[3].to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)

## 配属人数の確認

In [313]:
#配属される人数 スタッフコードで重複削除
df_N = df_predlist.copy().drop_duplicates(subset='スタッフコード')

#科目別人数を確認
df_N_jpn =df_N[df_N['科目']=='国語']
df_N_math =df_N[df_N['科目']=='数学']
df_N_eng =df_N[df_N['科目']=='英語']

In [314]:
#配置した人数
print(len(df_N), len(df_N_jpn), len(df_N_math), len(df_N_eng), len(df_test))

3793 1149 902 1742 3793


In [318]:
#配置したい人数
print('受領データ:3887', 'エラーデータ：94', '国語計画人数:1180', '数学計画人数919', '英語計画人数1787')

受領データ:3887 エラーデータ：94 国語計画人数:1180 数学計画人数919 英語計画人数1787


# メモ

In [306]:
def alg1(df_predlist, df_prilist):
    """
    優先度リストから、優先度の高い分野から順に、分野名と人数を参照する for enumerate
    予測値リストから、分野名でフィルタリングし、予測値でソートをして、高い順に必要人数を選択
    
    """
    df_pred = df_predlist.copy()#予測値リスト
    df_pri = df_prilist.copy()
    df_res=pd.DataFrame(columns=['スタッフコード', '科目', '分野', '学年', 'AI想定完全一致率'])
    """
    df_jpn = df_prilist[0].copy()#優先度リスト
    df_math = df_prilist[1].copy()#優先度リスト
    df_eng = df_prilist[2].copy()#優先度リスト
    """
    #科目を選択
    kamoku =[0,1,2]
    for k in kamoku:#科目を選択
        # for i in df_pri[k]['分野']:#分野名を選択
        for idx, value in enumerate(df_pri[k]['優先度']):#分野名を選択
            # print(idx)
            bunya = df_pri[k].at[idx,'分野']#分野
            gakunen = df_pri[k].at[idx,'学年']#学年
            ninzu = df_pri[k].at[idx,'配置希望人数']#配置希望人数
            # print(bunya, gakunen, ninzu)
            
            #予測値リストを優先度に従って分野名と学年でフィルタ
            df_temp = df_pred[(df_pred['分野']==bunya)&(df_pred['学年']==gakunen)]
            #予測値を降順で並び替え
            df_temp = df_temp.sort_values('AI想定完全一致率', ascending=False)
            #配置希望人数を取得
            df_temp =df_temp.iloc[:ninzu,:]
            #選択結果を結果のデータフレームに保存
            df_res = df_res.append(df_temp)
            #取得したスタッフコードを削除
            selected_staff = pd.Series(df_temp['スタッフコード'].unique())
            df_pred = df_pred[~(df_pred['スタッフコード'].isin(selected_staff))]#選択されたスタッフコードを削除
            
    return df_res

In [321]:
#学年分野別予測値の集計
def make_agg(df_input):
    df = df_input.copy()
    col_name = 'AI想定完全一致率'
    df_ret = df.groupby(['分野','学年'],as_index=False).agg(
        AI想定完全一致率_N数=pd.NamedAgg(column = col_name, aggfunc="count"),
        AI想定完全一致率_平均値=pd.NamedAgg(column = col_name, aggfunc="mean"),
        AI想定完全一致率_中央値=pd.NamedAgg(column = col_name, aggfunc="median"),
        AI想定完全一致率_最大値=pd.NamedAgg(column = col_name, aggfunc="max"),
        AI想定完全一致率_最小値=pd.NamedAgg(column = col_name, aggfunc="min"))
    return df_ret

In [331]:
#分野の表記を修正
def merge_prilist_aggresult(df_prilist, df_agg):
    df1 = df_prilist.copy()
    df2 = df_agg
    #優先度と配属人数の結合
    df_jpn = pandas_tool.merge(df1[0], df2, key = ["分野","学年"],how = 'left')
    df_math = pandas_tool.merge(df1[1], df2, key = ["分野","学年"],how = 'left')
    df_eng = pandas_tool.merge(df1[2], df2, key = ["分野","学年"],how = 'left')
    return [df_jpn, df_math, df_eng]